# Event Agent Test Notebook - Supabase + Discord

This notebook tests the complete event extraction workflow:
1. Scraping a URL
2. Saving to Supabase
3. Extracting event details
4. Creating Discord event

All stages are instrumented with Logfire for monitoring.


## Section 1: Setup Environment and Configure Logfire


In [ ]:
# 1.1 Load Environment Variables
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

print(f"✅ Python environment ready")
print(f"   Python version: {sys.version}")
print(f"   Working directory: {Path.cwd()}")

# Load .env file
load_dotenv()

print(f"\n✅ Environment variables loaded")
print(f"   OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Missing'}")
print(f"   DISCORD_BOT_TOKEN: {'✓ Set' if os.getenv('DISCORD_BOT_TOKEN') else '✗ Missing'}")
print(f"   SUPABASE_URL: {'✓ Set' if os.getenv('SUPABASE_URL') else '✗ Missing'}")
print(f"   SUPABASE_KEY: {'✓ Set' if os.getenv('SUPABASE_KEY') else '✗ Missing'}")


In [ ]:
# 1.2 Configure Logfire
import logfire
from mcp_ce.agentic_tools.logfire_config import configure_logfire

# Configure Logfire for instrumentation
logfire_configured = configure_logfire(
    send_to_logfire="if-token-present",
    enable=True
)

if logfire_configured:
    print("✅ Logfire configured successfully")
    print(f"   View traces at: https://logfire.pydantic.dev/")
else:
    print("⚠️ Logfire not configured (continuing without it)")


## Section 2: Test Individual Stages

### Stage 1: Test Web Scraping


In [ ]:
# Test scraping only
from mcp_ce.tools.crawl4ai.crawl_website import crawl_website

TEST_URL = "https://www.bluesmuse.dance/"

print(f"🌐 Testing scraping: {TEST_URL}")
print()

with logfire.span("test_scraping"):
    scrape_result = await crawl_website(
        url=TEST_URL,
        extract_images=False,
        extract_links=False,
        word_count_threshold=5,
        headless=True,
    )

if scrape_result.get("success"):
    content = scrape_result.get("content", {})
    print(f"✅ Scraping successful!")
    print(f"   Title: {content.get('title', 'N/A')}")
    print(f"   Content length: {content.get('content_length', 0)} characters")
    print(f"   Markdown preview (first 500 chars):")
    print(content.get("markdown", "")[:500])
    print("...")
else:
    print(f"❌ Scraping failed: {scrape_result.get('error', 'Unknown error')}")


### Stage 2: Test Supabase Storage


In [ ]:
# Test saving to Supabase (using scraped content from previous cell)
from mcp_ce.tools.supabase.add_document import add_document
from datetime import datetime

if scrape_result.get("success"):
    content = scrape_result.get("content", {})
    
    print(f"💾 Testing Supabase storage...")
    print()
    
    with logfire.span("test_supabase_storage"):
        supabase_result = await add_document(
            url=TEST_URL,
            title=content.get("title", TEST_URL),
            content=content.get("markdown", ""),
            description=content.get("description", ""),
            metadata={
                "source": "test_notebook",
                "scraped_at": datetime.now().isoformat(),
                "test_stage": "supabase_storage"
            },
            table_name="documents"
        )
    
    if supabase_result.is_success:
        doc = supabase_result.result
        print(f"✅ Saved to Supabase!")
        print(f"   Document ID: {doc.id}")
        print(f"   Title: {doc.title}")
        print(f"   Created at: {doc.created_at}")
        SUPABASE_DOC_ID = doc.id
    else:
        print(f"❌ Supabase save failed: {supabase_result.error}")
        SUPABASE_DOC_ID = None
else:
    print("⚠️ Skipping Supabase test - scraping failed in previous cell")


### Stage 3: Test Event Extraction


In [ ]:
# Test event extraction (using scraped content)
from mcp_ce.tools.events.evaluator_optimizer import extract_event_with_quality_control

if scrape_result.get("success"):
    content = scrape_result.get("content", {})
    content_markdown = content.get("markdown", "")
    
    print(f"🤖 Testing event extraction...")
    print()
    
    with logfire.span("test_event_extraction"):
        event, quality_score = await extract_event_with_quality_control(
            url=TEST_URL,
            content=content_markdown,
            quality_threshold=0.7,
            max_iterations=3
        )
    
    print(f"✅ Event extraction complete!")
    print(f"   Event Title: {event.title}")
    print(f"   Date: {event.date or 'N/A'}")
    print(f"   Start Time: {event.start_time or 'N/A'}")
    print(f"   Location: {event.get_full_location() or 'N/A'}")
    print(f"   Organizer: {event.organizer or 'N/A'}")
    print()
    print(f"📊 Quality Score:")
    print(f"   Overall: {quality_score.overall_score:.2f}")
    print(f"   Completeness: {quality_score.completeness:.2f}")
    print(f"   Accuracy: {quality_score.accuracy:.2f}")
    print(f"   Confidence: {quality_score.confidence:.2f}")
    print(f"   Acceptable: {quality_score.is_acceptable()}")
    if quality_score.issues:
        print(f"   Issues: {', '.join(quality_score.issues)}")
    
    EXTRACTED_EVENT = event
else:
    print("⚠️ Skipping extraction test - scraping failed in previous cell")
    EXTRACTED_EVENT = None


### Stage 4: Test Discord Event Creation


In [ ]:
# Test Discord event creation (using extracted event)
from mcp_ce.tools.discord.create_scheduled_event import create_scheduled_event
from mcp_ce.tools.discord.list_servers import list_servers
from mcp_ce.tools.discord._bot_helper import is_bot_ready

if EXTRACTED_EVENT:
    print(f"📅 Testing Discord event creation...")
    print()
    
    # Get server ID
    discord_server_id = os.getenv("DISCORD_SERVER_ID")
    
    if not discord_server_id:
        print("   No DISCORD_SERVER_ID in env, fetching available servers...")
        servers_result = await list_servers()
        if servers_result.is_success and servers_result.result:
            servers = servers_result.result.servers
            if servers:
                discord_server_id = servers[0]["id"]
                print(f"   Using server: {servers[0]['name']} (ID: {discord_server_id})")
    
    if not discord_server_id:
        print("❌ No Discord server available")
    elif not is_bot_ready():
        print("❌ Discord bot not ready")
    else:
        # Parse datetime
        start_dt, end_dt = EXTRACTED_EVENT.convert_datetime_for_discord()
        
        if not start_dt:
            print("❌ Could not parse event date/time")
        else:
            # Prepare description
            location = EXTRACTED_EVENT.get_full_location() or "See event details"
            if len(location) > 100:
                location = location[:97] + "..."
            
            description_parts = []
            if EXTRACTED_EVENT.organizer:
                description_parts.append(f"Host: {EXTRACTED_EVENT.organizer}")
            if EXTRACTED_EVENT.price:
                description_parts.append(f"Price: {EXTRACTED_EVENT.price}")
            if EXTRACTED_EVENT.description:
                desc = EXTRACTED_EVENT.description[:900] if len(EXTRACTED_EVENT.description) > 900 else EXTRACTED_EVENT.description
                description_parts.append(desc)
            if EXTRACTED_EVENT.url:
                description_parts.append(f"More info: {EXTRACTED_EVENT.url}")
            
            description = "\n\n".join(description_parts) if description_parts else None
            
            with logfire.span("test_discord_event_creation"):
                discord_result = await create_scheduled_event(
                    server_id=discord_server_id,
                    name=EXTRACTED_EVENT.title[:100],
                    start_time=start_dt.isoformat(),
                    end_time=end_dt.isoformat() if end_dt else None,
                    description=description,
                    entity_type="external",
                    location=location
                )
            
            if discord_result.is_success:
                event_result = discord_result.result
                print(f"✅ Discord event created!")
                print(f"   Event ID: {event_result.event_id}")
                print(f"   Event URL: {event_result.url}")
                print(f"   Name: {event_result.name}")
                DISCORD_EVENT_ID = event_result.event_id
                DISCORD_EVENT_URL = event_result.url
            else:
                print(f"❌ Discord event creation failed: {discord_result.error}")
                DISCORD_EVENT_ID = None
                DISCORD_EVENT_URL = None
else:
    print("⚠️ Skipping Discord test - event extraction failed in previous cell")


In [ ]:
# Test the complete agent workflow
from mcp_ce.agentic_tools.events.create_from_url_supabase import create_event_from_url_with_supabase

TEST_URL = "https://www.bluesmuse.dance/"

print(f"🚀 Testing complete workflow: {TEST_URL}")
print(f"   This will: scrape → save to Supabase → extract → create Discord event")
print()

with logfire.span("complete_workflow_test"):
    result = await create_event_from_url_with_supabase(
        url=TEST_URL,
        save_to_supabase=True,
        quality_threshold=0.7,
        max_iterations=3
    )

print(f"\n📊 Workflow Result:")
print(f"   Success: {result['success']}")
print()

if result["event"]:
    event = result["event"]
    print(f"✅ Event Extracted:")
    print(f"   Title: {event.title}")
    print(f"   Date: {event.date or 'N/A'}")
    print(f"   Time: {event.start_time or 'N/A'}")
    print(f"   Location: {event.get_full_location() or 'N/A'}")
    print()

if result.get("quality_score"):
    qs = result["quality_score"]
    print(f"📊 Quality Score:")
    print(f"   Overall: {qs['overall']:.2f}")
    print(f"   Acceptable: {qs['is_acceptable']}")
    print()

if result.get("supabase_document_id"):
    print(f"💾 Supabase:")
    print(f"   Document ID: {result['supabase_document_id']}")
    print()

if result.get("discord_event_id"):
    print(f"📅 Discord:")
    print(f"   Event ID: {result['discord_event_id']}")
    print(f"   Event URL: {result['discord_event_url']}")
    print()

if result.get("errors"):
    print(f"⚠️ Errors:")
    for error in result["errors"]:
        print(f"   - {error}")

print(f"\n💡 View detailed traces in Logfire: https://logfire.pydantic.dev/")


## Section 4: View Logfire Traces


In [ ]:
# Instructions for viewing Logfire traces
print("🔍 To view Logfire traces:")
print("   1. Visit https://logfire.pydantic.dev/")
print("   2. Select your project")
print("   3. Filter by span names:")
print("      - test_scraping")
print("      - test_supabase_storage")
print("      - test_event_extraction")
print("      - test_discord_event_creation")
print("      - complete_workflow_test")
print("   4. View execution times, errors, and detailed logs")
print()
print("💡 Each stage is instrumented with:")
print("   - Span timing")
print("   - Success/failure status")
print("   - Error messages")
print("   - Contextual metadata")
